In [1]:
# Speed comparison SZ
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20210112/logs_20210112_zs_96_03_day_96data.tar.gz/mdLog_SZ_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20210112/zs_6801_20210112_168.26.5.167.tar.gz/Logs_full/mdLog_SZ_***'))
assert(len(path1) == 1)
print(path1[0])
logSZ = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
logSZ1 = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSZ["source"].unique())
print("new data source type: ")
print(logSZ1["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())
assert(logSZ['source'].nunique() == 1)
assert(logSZ1['source'].nunique() == 1)

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

if (logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1) | \
(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1):
    print('cum_amount precision is insufficient')
    columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
else:
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
       "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
       "ask4q", "ask5q", "openPrice", "numTrades"]

data1 = logSZ[(logSZ["cum_volume"] > 0)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0)]

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    print(test[np.isnan(test["sequenceNo_y"])]['time_x'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_y"])]['StockID'].astype(int).unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    print(test[np.isnan(test["sequenceNo_x"])]['time_y'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_x"])]['StockID'].astype(int).unique())
    display(n2-n1)
    

import pandas as pd
import random
import numpy as np
import pickle

path = np.array(glob.glob('/mnt/data/20210112/logs_20210112_zs_96_03_day_96data.tar.gz/mdTradeLog***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20210112/zs_6801_20210112_168.26.5.167.tar.gz/Logs_full/mdTradeLog***'))
assert(len(path1) == 1)
print(path1[0])
path2 = np.array(glob.glob('/mnt/data/20210112/logs_20210112_zs_96_03_day_96data.tar.gz/mdOrderLog***'))
assert(len(path2) == 1)
print(path2[0])
path3 = np.array(glob.glob('/mnt/data/20210112/zs_6801_20210112_168.26.5.167.tar.gz/Logs_full/mdOrderLog***'))
assert(len(path3) == 1)
print(path3[0])

TradeLog = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum", "mdSource"]]
OrderLog = pd.read_csv(path2[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty", "mdSource"]]
TradeLog1 = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum", "mdSource"]]
OrderLog1= pd.read_csv(path3[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty", "mdSource"]]
stocks = list(set(OrderLog1["SecurityID"].unique()) & set(OrderLog["SecurityID"].unique()))
display(len(stocks))
OrderLogSZ = OrderLog[OrderLog["SecurityID"].isin(stocks)]
OrderLogSZ1 = OrderLog1[OrderLog1["SecurityID"].isin(stocks)]
del OrderLog
del OrderLog1
TradeLogSZ = TradeLog[TradeLog["SecurityID"].isin(stocks)]
TradeLogSZ1 = TradeLog1[TradeLog1["SecurityID"].isin(stocks)]
del TradeLog
del TradeLog1
OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))
print(len(OrderLogSZ["SecurityID"].unique()))
print(len(OrderLogSZ1["SecurityID"].unique()))
print(len(TradeLogSZ["SecurityID"].unique()))
print(len(TradeLogSZ1["SecurityID"].unique()))
print(OrderLogSZ["mdSource"].unique())
print(OrderLogSZ1["mdSource"].unique())
print(TradeLogSZ["mdSource"].unique())
print(TradeLogSZ1["mdSource"].unique())
print(OrderLogSZ["exchId"].unique())
print(OrderLogSZ1["exchId"].unique())
print(TradeLogSZ["exchId"].unique())
print(TradeLogSZ1["exchId"].unique())

TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

# TradeLogSZ = TradeLogSZ[TradeLogSZ['TransactTime'] >= TradeLogSZ1['TransactTime'].min()]
columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
# columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradePrice", "TradeQty", "BidApplSeqNum","OfferApplSeqNum"]

# TradeLogSZ1 = TradeLogSZ1.drop_duplicates(columns, keep='first')

re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)
    
    

columns = ["TransactTime", "ApplSeqNum","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]

# OrderLogSZ = OrderLogSZ[OrderLogSZ['TransactTime'] >= OrderLogSZ1['TransactTime'].min()]
ree = pd.merge(OrderLogSZ, OrderLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = ree["sequenceNo_x"].count()
n2 = ree["sequenceNo_y"].count()
len1 = len(ree)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(ree[np.isnan(ree["sequenceNo_y"])])
    print(len(ree[np.isnan(ree["sequenceNo_y"])]))
    print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("test is complete, baseline is not complete:")
    display(ree[np.isnan(ree["sequenceNo_x"])])
    print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)


/mnt/data/20210112/logs_20210112_zs_96_03_day_96data.tar.gz/mdLog_SZ_20210112_0847.csv
/mnt/data/20210112/zs_6801_20210112_168.26.5.167.tar.gz/Logs_full/mdLog_SZ_20210112_0905.csv
original data source type: 
[24]
new data source type: 
[31]
['08:47:54.000' '08:48:54.000' '08:49:54.000' ... '15:28:03.000'
 '15:29:03.000' '15:30:03.000']
['09:05:54.000' '09:06:54.000' '09:07:54.000' ... '15:44:03.000'
 '15:45:03.000' '15:46:03.000']


array([1, 2, 6, 7])

array([1])

cum_amount precision is insufficient
2334
2127
7042779
7044819
7044819
-----------------------------------------------


'baseline is not complete:'

...

2127

2127
2127
2127
2127
[24]
[31]
[24]
[31]
[2]
[2]
[2]
[2]
45199207
45199207
45199207
-----------------------------------------------
50309670
50309668
50309670
-----------------------------------------------


'test is not complete:'

,clockAtArrival_x,sequenceNo_x,exchId_x,TransactTime,ApplSeqNum,SecurityID,Side,OrderType,Price,OrderQty,mdSource_x,clockAtArrival_y,sequenceNo_y,exchId_y,mdSource_y
121870,1610414102879263,570217,2,91500470,35765,155,2,2,95600,5700,24,NaN,NaN,NaN,NaN
121871,1610414102879264,570218,2,91500470,35766,728,2,2,82500,66000,24,NaN,NaN,NaN,NaN


2
[91500470]
2
[155 728]
